In [1]:
import pandas as pd
import numpy as np
import random
import math
import json
from datetime import timedelta as td
from datetime import time
from datetime import datetime
from matplotlib.pyplot import plot
from matplotlib import pyplot as plt

import sys
sys.path.insert(1, '../DRTsim')
import sim_utils, const
from sim_utils import Activity, Coord
from const import ActivityType
from shapely.geometry import Point
import shapely.wkt

In [2]:
def draw_population(df, weights):
    df2 = df.loc[np.repeat(df.index.values, weights)]
    return df2

# Time distribution

In [3]:
resfil_raw = pd.read_csv('/home/ai6644/Malmo/Data/Sjöbo/Travel_survey_Skane_2013/RVU_resfil.csv')
# resfil_raw = resfil_raw.loc[:, ['rf2_starttid','individvikt', 'ärende_2', 'Id', 'rf6_ankomsttid']]
resfil_raw = resfil_raw[resfil_raw['dagtyp'] == 1]

In [4]:
resfil_raw.Id.unique()

array([1000074., 1000116., 1000207., ..., 1599927., 1599968., 1599976.])

In [5]:
trip_chains = {}
for pid in resfil_raw.Id.unique():
    trips = resfil_raw[resfil_raw.Id == pid]
    chain = tuple(trips['rf3a_ärende'].to_list())
    
    if any([math.isnan(x) for x in chain]):
        chain = tuple(9.0 if math.isnan(x) else x for x in chain)
    
    if chain in trip_chains.keys():
        trip_chains[chain] += trips['individvikt'].iloc[0]
    else:
        trip_chains[chain] = trips['individvikt'].iloc[0]

In [6]:
trip_chains

{(8.0, 1.0): 9825.197574257909,
 (4.0, 4.0, 4.0, 1.0): 450.41171288300006,
 (7.0, 11.0, 11.0, 1.0, 6.0, 1.0): 10.434357633,
 (5.0, 1.0, 5.0, 1.0): 1730.1993438479005,
 (2.0, 1.0): 148178.48504147027,
 (6.0, 1.0): 17557.09212565871,
 (6.0, 13.0): 203.72043653629999,
 (14.0, 1.0): 1956.5495930138,
 (16.0, 13.0, 6.0, 1.0): 61.311805014,
 (2.0, 1.0, 5.0, 5.0): 221.84067497900003,
 (5.0, 1.0, 6.0, 5.0, 7.0): 56.43792818,
 (6.0, 1.0, 7.0, 1.0): 1088.7908691604,
 (3.0, 2.0, 8.0, 1.0): 182.26363451,
 (8.0, 1.0, 12.0): 61.9765775029,
 (1.0, 7.0, 1.0, 11.0, 1.0, 2.0, 1.0): 78.320372129,
 (5.0, 2.0, 1.0): 5514.4060120517015,
 (2.0, 6.0, 1.0, 2.0, 1.0): 114.91870662000001,
 (2.0, 5.0, 1.0, 2.0, 1.0): 9.5929977281,
 (6.0, 15.0, 1.0): 22.8384550319,
 (9.0, 1.0): 3685.3127575392004,
 (2.0, 16.0, 16.0, 2.0, 1.0, 13.0, 13.0, 13.0): 69.91916196,
 (7.0, 1.0, 6.0, 14.0, 1.0): 15.201725425,
 (6.0, 10.0, 13.0, 1.0): 34.60471488,
 (9.0, 2.0, 6.0, 7.0, 7.0, 7.0, 7.0, 7.0, 1.0): 56.454012124,
 (12.0, 1.0): 322

In [27]:
freq = pd.DataFrame.from_dict(trip_chains, orient='index')

In [28]:
freq

,0
"(8.0, 1.0)",9825.197574
"(4.0, 4.0, 4.0, 1.0)",450.411713
"(7.0, 11.0, 11.0, 1.0, 6.0, 1.0)",10.434358
"(5.0, 1.0, 5.0, 1.0)",1730.199344
"(2.0, 1.0)",148178.485041
...,...
"(9.0, 1.0, 88.0, 17.0, 1.0)",22.339952
"(13.0, 12.0, 14.0, 88.0, 1.0)",19.971822
"(5.0, 2.0, 5.0, 16.0, 1.0)",46.427189
"(5.0, 5.0, 2.0, 5.0, 5.0, 5.0, 1.0)",20.329177


In [9]:
freq[0].sum()

583713.6644216803

In [10]:
freq['pers'] = freq[0]/freq[0].sum()

In [11]:
codes = {1.0: 'work',
         2.0: 'education',
         3.0: 'tjänst',
         4.0: 'hämta barn or other',
         5.0: 'inköp',
         6.0: 'hospital',
         7.0: 'motion/forening',
         8.0: 'friends or pleasure',
         9.0: 'other',
         10.0: 'home',
         -1: 'none'
        }

In [13]:
new_ind = []
def denan(x):
    if math.isnan(x):
        return -1
    else:
        return x

for ind in freq.index:
    temp = tuple([denan(i) for i in ind])
    new_i = tuple([codes[i] for i in temp])
    new_ind.append(new_i)

KeyError: 11.0

In [14]:
new_ind

[('friends or pleasure', 'work'),
 ('hämta barn or other', 'hämta barn or other', 'hämta barn or other', 'work')]

In [15]:
freq.index = new_ind

ValueError: Length mismatch: Expected axis has 3115 elements, new values have 2 elements

In [16]:
freq.sort_values(0, ascending=False).iloc[0:50]

,0,pers
"(2.0, 1.0)",148178.485041,0.253855
"(4.0, 1.0)",41740.672737,0.071509
"(6.0, 1.0)",17557.092126,0.030078
"(8.0, 1.0)",9825.197574,0.016832
"(7.0, 1.0)",9819.089272,0.016822
"(10.0, 1.0)",8365.725890,0.014332
"(2.0, 1.0, 10.0, 1.0)",7167.314840,0.012279
"(13.0, 1.0)",6510.234875,0.011153
"(11.0, 1.0)",6331.031579,0.010846
"(2.0, 6.0, 1.0)",6194.635397,0.010612


In [17]:
sum([2.0 in x for x in freq.index] * freq.pers)

0.5438069911604992

In [18]:
sum([4.0 in x for x in freq.index] * freq.pers)

0.14504962259960408

In [19]:
sum([((2.0 in x) and (4.0 in x)) for x in freq.index] * freq.pers)

0.0100554912889137

In [20]:
sum([1.0 in x for x in freq.index] * freq.pers)

0.9483250588330376

In [21]:
sum([len(x) in x for x in freq.index] * freq.pers)

0.3555945607055996

In [29]:
freq.sort_values(0, ascending=False).iloc[0:50]['pers'].sum()

KeyError: 'pers'

In [55]:
freq_pairs = {}
freq_dest = {}
for ind, rec in freq.iterrows():
    for s,f in zip([1] + [i for i in ind[0:-1]], ind):
        if (s,f) in freq_pairs.keys():
            freq_pairs[s,f] += rec[0]
            freq_dest[f] += rec[0]
        else:
            freq_pairs[s,f] = rec[0]
            freq_dest[f] = rec[0]

In [56]:
freq_dest = pd.DataFrame.from_dict(freq_dest, orient='index')

In [60]:
freq_dest[0] = freq_dest[0]/freq_dest[0].sum()
freq_dest.sort_values(by=0, ascending=False)

,0
1.0,0.582662
4.0,0.047571
10.0,0.047504
7.0,0.045042
11.0,0.034751
13.0,0.033991
8.0,0.032396
6.0,0.031847
2.0,0.031011
9.0,0.029812


In [40]:
freq_pairs = pd.DataFrame.from_dict(freq_pairs, orient='index')

In [36]:
freq_pairs['pers'] = freq_pairs[0]/freq_pairs[0].sum()

In [37]:
freq_pairs.sort_values(0, ascending=False).iloc[0:50]

,0,pers
"(1, 2.0)",288902.029516,0.154500
"(2.0, 1.0)",238634.793208,0.127618
"(1, 4.0)",79481.714645,0.042505
"(6.0, 1.0)",77357.688331,0.041369
"(4.0, 1.0)",68024.982178,0.036379
"(1.0, 6.0)",63118.283315,0.033755
"(1, 5.0)",62625.651861,0.033491
"(10.0, 1.0)",54529.915313,0.029162
"(1, 10.0)",51732.722228,0.027666
"(5.0, 1.0)",48385.830626,0.025876


In [58]:
freq_pairs.sort_values(0, ascending=False).iloc[0:30]['pers'].sum()

0.9011451012612556

In [59]:
for i in range(1,18):
    print(i, resfil_raw[resfil_raw['rf3a_ärende']==i]['individvikt'].sum()/resfil_raw['individvikt'].sum())
for i in range(1,18):
    print(i, resfil_raw[resfil_raw['rf3a_ärende']==i]['individvikt'].sum())

1 0.3908378653238448
2 0.16442440549371576
3 0.034497172475099105
4 0.042663525000152726
5 0.047403499720879295
6 0.0632193708209627
7 0.0406607961382408
8 0.018379354681719254
9 0.009435475005004416
10 0.04227486409602335
11 0.02870767898985723
12 0.03214818531413418
13 0.050038863496353225
14 0.008071117268218774
15 0.0018983266895951317
16 0.009444825823925153
17 0.0020838104688969206
1 928769.8359654546
2 390730.89295628923
3 81977.55658710751
4 101383.7159387495
5 112647.5824533255
6 150231.7197914614
7 96624.519551726
8 43675.886467006196
9 22422.0459432528
10 100460.1193371886
11 68219.6600482045
12 76395.5272758824
13 118910.1444991706
14 19179.846494786798
15 4511.099677210699
16 22444.2668162175
17 4951.875135682801


In [60]:
resfil_raw['individvikt'].sum()

2376355.820068468

# Age to different groups

In [19]:
resfil_raw.alder_tab

0        81.0
1        81.0
2        15.0
3        15.0
4        15.0
         ... 
56327    68.0
56328    68.0
56329    42.0
56330    42.0
56331    42.0
Name: alder_tab, Length: 56332, dtype: float64

In [ ]:
'Ald0_6','Ald7_15','Ald16_19','Ald20_24','Ald25_44','Ald45_64','Ald65_W'

In [27]:
bins = [0, 6, 15, 19, 24, 44, 64, 999]
labels = ['Ald0_6','Ald7_15','Ald16_19','Ald20_24','Ald25_44','Ald45_64','Ald65_W']

In [31]:
resfil_raw['age_scb'] = pd.cut(resfil_raw.alder_tab, bins=bins, labels=labels)

In [32]:
resfil_raw

,Id,Id_time,förfl_nr,lillam,lillan,storaN,Stratum,mätdag,dagtyp,dagtyp_2,...,kl6_km,bostad_storst_småort_landsby,restyp,individvikt,individvikt_svarandemängd,urval_trompNr,rf1_TrompNr,rf4_TrompNr,filter_$,age_scb
0,1000074.0,10000741.0,1.0,519.0,1608.0,7845.0,27.0,1.0,1.0,1.0,...,4.0,2.0,2.0,8.330817,0.201277,NaN,NaN,NaN,NaN,Ald65_W
1,1000074.0,10000742.0,2.0,519.0,1608.0,7845.0,27.0,1.0,1.0,1.0,...,4.0,2.0,2.0,8.330817,0.201277,NaN,NaN,NaN,NaN,Ald65_W
2,1000116.0,10001161.0,1.0,2118.0,6066.0,25563.0,22.0,1.0,1.0,1.0,...,3.0,3.0,2.0,22.281823,0.538341,NaN,NaN,NaN,NaN,Ald7_15
3,1000116.0,10001162.0,2.0,2118.0,6066.0,25563.0,22.0,1.0,1.0,1.0,...,2.0,3.0,2.0,22.281823,0.538341,NaN,NaN,NaN,NaN,Ald7_15
4,1000116.0,10001163.0,3.0,2118.0,6066.0,25563.0,22.0,1.0,1.0,1.0,...,2.0,3.0,2.0,22.281823,0.538341,NaN,NaN,NaN,NaN,Ald7_15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56327,1556554.0,155655410.0,10.0,3123.0,10054.0,219976.0,57.0,5.0,1.0,1.0,...,NaN,1.0,2.0,35.289377,0.852610,1.0,NaN,1.0,0.0,Ald65_W
56328,1556554.0,155655411.0,11.0,3123.0,10054.0,219976.0,57.0,5.0,1.0,1.0,...,NaN,1.0,2.0,35.289377,0.852610,1.0,1.0,1.0,0.0,Ald65_W
56329,1623594.0,162359410.0,10.0,290.0,780.0,10567.0,42.0,6.0,2.0,2.0,...,4.0,3.0,2.0,42.948336,1.037655,NaN,NaN,NaN,NaN,Ald25_44
56330,1623594.0,162359411.0,11.0,290.0,780.0,10567.0,42.0,6.0,2.0,2.0,...,4.0,3.0,2.0,42.948336,1.037655,NaN,NaN,NaN,NaN,Ald25_44


# And now grouped by age

In [71]:
freq_age = []
for age in labels:
    trip_chains = {}
    for pid in resfil_raw[resfil_raw.age_scb == age].Id.unique():
        trips = resfil_raw[resfil_raw.Id == pid]
        chain = tuple(trips['ärende_2'].to_list())
        
        if any([math.isnan(x) for x in chain]):
            chain = tuple(9.0 if math.isnan(x) else x for x in chain)
        
        if chain in trip_chains.keys():
            trip_chains[chain] += trips['individvikt'].iloc[0]
        else:
            trip_chains[chain] = trips['individvikt'].iloc[0]
    freq_age.append(pd.DataFrame.from_dict(trip_chains, orient='index',columns=[age]))

In [69]:
freq_age = freq_age[0].join(freq_age[1], how='outer').join(freq_age[2], how='outer').join(freq_age[3], how='outer').join(freq_age[4], how='outer').fillna(0)
freq_age = freq_age.fillna(0)

In [70]:
freq_age

,Ald0_6,Ald7_15,Ald16_19,Ald20_24,Ald25_44
"(1.0,)",0,0.0,0.0,73.417499,435.522265
"(1.0, 1.0)",0,0.0,0.0,0.000000,45.735289
"(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 10.0)",0,0.0,0.0,0.000000,194.675958
"(1.0, 1.0, 1.0, 1.0, 4.0, 4.0, 10.0)",0,0.0,0.0,0.000000,144.433471
"(1.0, 1.0, 1.0, 1.0, 10.0)",0,0.0,0.0,0.000000,222.994266
...,...,...,...,...,...
"(10.0, 8.0, 10.0, 5.0, 10.0)",0,0.0,0.0,0.000000,17.688901
"(10.0, 8.0, 10.0, 9.0, 10.0)",0,0.0,0.0,0.000000,34.983895
"(10.0, 9.0)",0,0.0,0.0,104.971425,0.000000
"(10.0, 10.0)",0,0.0,0.0,0.000000,46.633854


In [60]:
freq_age_perc = (freq_age/freq_age.sum()).fillna(0)

In [61]:
freq_age_perc

,Ald0_6,Ald7_15,Ald16_19,Ald20_24,Ald25_44
"(1.0,)",0.0,0.0,0.0,0.001204,0.001647
"(1.0, 1.0)",0.0,0.0,0.0,0.000000,0.000173
"(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 10.0)",0.0,0.0,0.0,0.000000,0.000736
"(1.0, 1.0, 1.0, 1.0, 4.0, 4.0, 10.0)",0.0,0.0,0.0,0.000000,0.000546
"(1.0, 1.0, 1.0, 1.0, 10.0)",0.0,0.0,0.0,0.000000,0.000843
...,...,...,...,...,...
"(10.0, 8.0, 10.0, 5.0, 10.0)",0.0,0.0,0.0,0.000000,0.000067
"(10.0, 8.0, 10.0, 9.0, 10.0)",0.0,0.0,0.0,0.000000,0.000132
"(10.0, 9.0)",0.0,0.0,0.0,0.001721,0.000000
"(10.0, 10.0)",0.0,0.0,0.0,0.000000,0.000176


In [62]:
freq_age_perc[1].sort_values(ascending=False)

KeyError: 1

In [63]:
freq_age_perc[2].sort_values(ascending=False)

KeyError: 2

In [64]:
freq_age_perc[3].sort_values(ascending=False)

KeyError: 3

In [65]:
freq_age_perc

,Ald0_6,Ald7_15,Ald16_19,Ald20_24,Ald25_44
"(1.0,)",0.0,0.0,0.0,0.001204,0.001647
"(1.0, 1.0)",0.0,0.0,0.0,0.000000,0.000173
"(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 10.0)",0.0,0.0,0.0,0.000000,0.000736
"(1.0, 1.0, 1.0, 1.0, 4.0, 4.0, 10.0)",0.0,0.0,0.0,0.000000,0.000546
"(1.0, 1.0, 1.0, 1.0, 10.0)",0.0,0.0,0.0,0.000000,0.000843
...,...,...,...,...,...
"(10.0, 8.0, 10.0, 5.0, 10.0)",0.0,0.0,0.0,0.000000,0.000067
"(10.0, 8.0, 10.0, 9.0, 10.0)",0.0,0.0,0.0,0.000000,0.000132
"(10.0, 9.0)",0.0,0.0,0.0,0.001721,0.000000
"(10.0, 10.0)",0.0,0.0,0.0,0.000000,0.000176
